# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [1]:
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="./6-2_content/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

/home/youngwon/anaconda3/envs/nlp_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-16 15:29:57.142328: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 15:29:57.639214: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


downstream_model_checkpoint_fpath: ./6-2_content/nlpbook/checkpoint-ner/epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: ./6-2_content/nlpbook/checkpoint-ner/label_map.txt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [2]:
import torch
from transformers import BertConfig, BertForTokenClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForTokenClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 레이블 맵 작성

범주 인덱스를 범주명과 매칭하는 사전을 만듭니다.

In [4]:
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [6]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
    return {
        "sentence": sentence,
        "result": result,
    }

In [10]:
inference_fn("3월 27일 3시에 김인성은 강남구에서 3000원을 내고 BBQ에서 치킨을 사 먹었다.")

{'sentence': '3월 27일 3시에 김인성은 강남구에서 3000원을 내고 BBQ에서 치킨을 사 먹었다.',
 'result': [{'token': '3월', 'predicted_tag': '날짜', 'top_prob': '0.9982'},
  {'token': '27', 'predicted_tag': '날짜', 'top_prob': '0.998'},
  {'token': '##일', 'predicted_tag': '날짜', 'top_prob': '0.9991'},
  {'token': '3', 'predicted_tag': 'O', 'top_prob': '0.9829'},
  {'token': '##시에', 'predicted_tag': '시간', 'top_prob': '0.7908'},
  {'token': '김', 'predicted_tag': '인명', 'top_prob': '0.9793'},
  {'token': '##인', 'predicted_tag': '인명', 'top_prob': '0.992'},
  {'token': '##성은', 'predicted_tag': '인명', 'top_prob': '0.9964'},
  {'token': '강남구', 'predicted_tag': '지명', 'top_prob': '0.9026'},
  {'token': '##에서', 'predicted_tag': 'O', 'top_prob': '0.9977'},
  {'token': '3000', 'predicted_tag': '통화', 'top_prob': '0.9984'},
  {'token': '##원을', 'predicted_tag': '통화', 'top_prob': '0.9768'},
  {'token': '내고', 'predicted_tag': 'O', 'top_prob': '0.9935'},
  {'token': 'B', 'predicted_tag': '기관명', 'top_prob': '0.8379'},
  {'token': '##B', 'pre